In [1]:
#importing libraries
import pandas as pd
import os
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.layers import Dense, Conv1D, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import StratifiedKFold
from keras.backend import dropout
import keras
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers

2024-08-04 19:08:55.561451: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-04 19:08:56.196272: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-08-04 19:08:56.196342: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-08-04 19:08:56.196347: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
# Set a seed value
seed_value= 3

# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
os.environ['PYTHONHASHSEED']=str(seed_value)

# 2. Set `python` built-in pseudo-random generator at a fixed value
random.seed(seed_value)

# 3. Set `numpy` pseudo-random generator at a fixed value
np.random.seed(seed_value)

# 4. Set `tensorflow` pseudo-random generator at a fixed value
tf.random.set_seed(seed_value)

tf.keras.utils.set_random_seed(seed_value)

# 5. For layers that introduce randomness like dropout, make sure to set seed values 
# model.add(Dropout(0.25, seed=seed_value))

In [3]:
#import relevant datasets
X_train = pd.read_csv("finalXtrain.csv")
y_train = pd.read_csv("finalYtrain.csv")
X_test = pd.read_csv("finalXtest.csv")
y_test = pd.read_csv("finalYtest.csv")
X_val = pd.read_csv("finalXval.csv")
y_val = pd.read_csv("finalYval.csv")

In [4]:
#check shape
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)
print(X_val.shape)
print(y_val.shape)

(221, 72)
(221, 3)
(47, 72)
(47, 3)
(48, 72)
(48, 3)


In [5]:
#build model

def define_model(seedval):
    inputs = tf.keras.Input(shape=(72, 1), name='input')
    x = Conv1D(filters=5, kernel_size=3,strides =3)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Conv1D(filters=45, kernel_size=5,strides =2)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation(tf.keras.activations.relu)(x)
    x = tf.keras.layers.MaxPooling1D(2)(x)
    x = Flatten()(x)
    x = Dropout(0.5,seed=seedval)(x)
    x = Dense(20,activation ='relu')(x)
#     x = Dropout(0.5,seed=seedval)(x)
#     x = Dense(10,activation ='relu')(x)
#     x = Dense(5,activation ='relu')(x)
    outputs = Dense(3, activation='softmax', name='predictions')(x)
    cnn_model = tf.keras.Model(inputs=inputs, outputs=outputs)
    cnn_model.compile(loss='categorical_crossentropy', optimizer= Adam(), metrics=['accuracy'])
    return cnn_model

In [6]:
#earlystopping
es = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=50)
checkpoint_path = 'alaarglys_model3.h5'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                   monitor='val_accuracy',
                                   save_best_only=True,
                                   mode='max',
                                   verbose=1)

In [7]:
model = define_model(seed_value)
print(model.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 72, 1)]           0         
                                                                 
 conv1d (Conv1D)             (None, 24, 5)             20        
                                                                 
 batch_normalization (BatchN  (None, 24, 5)            20        
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 24, 5)             0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 12, 5)            0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 4, 45)             1170  

2024-08-04 19:08:56.941924: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-08-04 19:08:56.941976: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (VLSI-CAD): /proc/driver/nvidia/version does not exist
2024-08-04 19:08:56.943104: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [8]:

history = model.fit(X_train, y_train,
              batch_size=10,
              verbose=1,
              epochs=500,
              validation_data = (X_val,y_val), 
              callbacks = [es,model_checkpoint]
                    ,class_weight = {0:2.4,1:2.23,2:7.37}
                   )

Epoch 1/500
 1/23 [>.............................] - ETA: 19s - loss: 6.3770 - accuracy: 0.1000
Epoch 1: val_accuracy improved from -inf to 0.43750, saving model to alaarglys_model3.h5
23/23 [==============================] - 1s 11ms/step - loss: 4.5582 - accuracy: 0.3665 - val_loss: 1.0840 - val_accuracy: 0.4375
Epoch 2/500
 1/23 [>.............................] - ETA: 0s - loss: 6.1550 - accuracy: 0.4000
Epoch 2: val_accuracy did not improve from 0.43750
23/23 [==============================] - 0s 3ms/step - loss: 3.9489 - accuracy: 0.3801 - val_loss: 1.1079 - val_accuracy: 0.3333
Epoch 3/500
 1/23 [>.............................] - ETA: 0s - loss: 4.0137 - accuracy: 0.2000
Epoch 3: val_accuracy did not improve from 0.43750
23/23 [==============================] - 0s 3ms/step - loss: 3.8072 - accuracy: 0.3982 - val_loss: 1.0953 - val_accuracy: 0.3750
Epoch 4/500
 1/23 [>.............................] - ETA: 0s - loss: 2.8185 - accuracy: 0.6000
Epoch 4: val_accuracy did not improve fr

Epoch 30/500
 1/23 [>.............................] - ETA: 0s - loss: 2.6293 - accuracy: 0.6000
Epoch 30: val_accuracy did not improve from 0.58333
23/23 [==============================] - 0s 2ms/step - loss: 2.9215 - accuracy: 0.5158 - val_loss: 1.0511 - val_accuracy: 0.5625
Epoch 31/500
 1/23 [>.............................] - ETA: 0s - loss: 2.7039 - accuracy: 0.6000
Epoch 31: val_accuracy did not improve from 0.58333
23/23 [==============================] - 0s 2ms/step - loss: 2.5687 - accuracy: 0.6018 - val_loss: 1.0346 - val_accuracy: 0.5625
Epoch 32/500
 1/23 [>.............................] - ETA: 0s - loss: 3.4274 - accuracy: 0.4000
Epoch 32: val_accuracy did not improve from 0.58333
23/23 [==============================] - 0s 2ms/step - loss: 2.7594 - accuracy: 0.4977 - val_loss: 1.0710 - val_accuracy: 0.5208
Epoch 33/500
 1/23 [>.............................] - ETA: 0s - loss: 1.0989 - accuracy: 0.9000
Epoch 33: val_accuracy did not improve from 0.58333
23/23 [==============

 1/23 [>.............................] - ETA: 0s - loss: 2.0227 - accuracy: 0.9000
Epoch 59: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 2ms/step - loss: 2.1851 - accuracy: 0.6380 - val_loss: 1.1296 - val_accuracy: 0.5208
Epoch 60/500
 1/23 [>.............................] - ETA: 0s - loss: 1.3866 - accuracy: 0.6000
Epoch 60: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 2ms/step - loss: 1.9321 - accuracy: 0.6878 - val_loss: 1.0880 - val_accuracy: 0.5833
Epoch 61/500
 1/23 [>.............................] - ETA: 0s - loss: 3.6607 - accuracy: 0.7000
Epoch 61: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 2ms/step - loss: 2.0933 - accuracy: 0.6652 - val_loss: 1.0753 - val_accuracy: 0.6250
Epoch 62/500
 1/23 [>.............................] - ETA: 0s - loss: 1.3301 - accuracy: 0.9000
Epoch 62: val_accuracy did not improve from 0.64583
23/23 [===========================

Epoch 89/500
 1/23 [>.............................] - ETA: 0s - loss: 1.9493 - accuracy: 0.5000
Epoch 89: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 3ms/step - loss: 1.8976 - accuracy: 0.7104 - val_loss: 1.2285 - val_accuracy: 0.5625
Epoch 90/500
 1/23 [>.............................] - ETA: 0s - loss: 1.1913 - accuracy: 0.8000
Epoch 90: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 2ms/step - loss: 1.6931 - accuracy: 0.7330 - val_loss: 1.2475 - val_accuracy: 0.5833
Epoch 91/500
 1/23 [>.............................] - ETA: 0s - loss: 1.0398 - accuracy: 0.8000
Epoch 91: val_accuracy did not improve from 0.64583
23/23 [==============================] - 0s 2ms/step - loss: 1.7060 - accuracy: 0.7557 - val_loss: 1.2778 - val_accuracy: 0.5625
Epoch 92/500
 1/23 [>.............................] - ETA: 0s - loss: 1.0849 - accuracy: 0.7000
Epoch 92: val_accuracy did not improve from 0.64583
23/23 [==============

In [9]:
model1 = tf.keras.models.load_model('alaarglys_model3.h5')
model1.evaluate(X_test,y_test)

2/2 [==============================] - 0s 3ms/step - loss: 0.8820 - accuracy: 0.5957


[0.8819532990455627, 0.5957446694374084]

In [10]:
y=model1.predict(X_test)

2/2 [==============================] - 0s 2ms/step


In [11]:
#class-wise accuracy
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test.to_numpy().argmax(axis=1), y.argmax(axis=1))
matrix.diagonal()/matrix.sum(axis=1)

array([0.8       , 0.52380952, 0.16666667])

In [12]:
model1.evaluate(X_val,y_val)

2/2 [==============================] - 0s 2ms/step - loss: 1.0487 - accuracy: 0.6458


[1.0486971139907837, 0.6458333134651184]

In [13]:
model1.evaluate(X_train,y_train)

7/7 [==============================] - 0s 1ms/step - loss: 0.5986 - accuracy: 0.7828


[0.5985991954803467, 0.7828054428100586]